In [ ]:
# CUDAメモリ断片化対策（OOM対策・torchより前に設定）
import os
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")

from pathlib import Path
import numpy as np
import pprint
import json
import torch
import gc
import traceback
import time
from scipy.io import savemat
from datetime import datetime
from myTools.output_converter import convert_to_matlab_files
from kilosort.run_kilosort import load_sorting
from probeinterface.io import write_probeinterface

from myTools.read_simulation import get_simulation_path
from myTools.set_gain_and_offset import set_gain_and_offset
from myTools.init_run import get_recording, get_probe, get_probe_sorted, set_probe_info, get_stimtime

import spikeinterface.full as si
# import spikeinterface.extractors as se
import spikeinterface.preprocessing as spre
import spikeinterface.sorters as ss
# import spikeinterface.postprocessing as spost
# import spikeinterface.qualitymetrics as sqm
# import spikeinterface.comparison as sc
import spikeinterface.exporters as sexp
# import spikeinterface.curation as scur
# import spikeinterface.widgets as sw

from kilosort.plots import plot_drift_amount, plot_drift_scatter, plot_diagnostics, plot_spike_positions


In [ ]:
dir_info = {
    "root_dir": r"C:\Users\tanaka-users\tlab\yasui\simulations",
    "ep": "001",
    "cd": "001",
}
dict_path = get_simulation_path(dir_info)

recording = si.load(dict_path["recording"])
probe = recording.get_probe()

### Setting sorters ###
do_preprocess = False
do_runsort = True
do_export_phy = True
sorter = "dartsort"

# DARTsortのデフォルトパラメータを取得
sort_params = ss.get_default_sorter_params("dartsort")

name_thisparam = "setting1"

sort_params["featurization_radius_um"] = 50

In [ ]:
### Preprocess recording ###
if do_preprocess:

    print("\n" + "="*20 + " parameter " + "="*20)
    print(f"\n{'='*5} {sorter} {'='*5}")
    pprint.pprint(sort_params)

    params_file = dict_path["exp"] / sorter / name_thisparam / "params.txt"
    params_file.parent.mkdir(parents=True, exist_ok=True)
    with open(str(params_file), "w", encoding="utf-8") as f:
        json.dump(sort_params, f, indent=4, ensure_ascii=False)

    print("="*5, sorter, "="*5)
    folder = dict_path["exp"] / sorter / name_thisparam
    pp_rec_folder = folder / "pp_rec"

    # DARTsort用前処理（NaN/Inf を避けるためmean common referenceを使用）
    print(f"  バンドパスフィルタ適用中...")
    recording_f = spre.bandpass_filter(recording, freq_min=300, freq_max=3000)
    
    # medianではなくmean common referenceを使用（NaN生成を回避）
    # medianは同じ値が続く場合や極端な値がある場合にNaNを生成する可能性がある
    print(f"  Common reference適用中... (mean)")
    try:
        recording_cmr = spre.common_reference(recording_f, reference="global", operator="mean")
    except Exception as e:
        print(f"  Common reference (mean) でエラー: {e}")
        print(f"  フォールバック: median common referenceを使用")
        recording_cmr = spre.common_reference(recording_f, reference="global", operator="median")
    
    # 前処理recordingを保存（NaN/Infチェックは後で実行）
    recording_preprocessed = recording_cmr.save(format="binary", folder=pp_rec_folder, overwrite=True)
else:
    print("skip preprocessing.")


In [ ]:
### Run sorting ###
if do_runsort:
    print(f"{'='*5} {sorter} {'='*5}")
    print(f"  {sorter}実行開始: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    try:
        start_time = time.time()
        
        sorter_output_dir = dict_path["exp"] / sorter / name_thisparam / "sorting"
        sorting = ss.run_sorter(
            sorter_name=sorter,
            folder=sorter_output_dir, 
            remove_existing_folder=True, 
            recording=recording,
            verbose=True,
            **sort_params
            )
            
        elapsed_time = time.time() - start_time
        print(f"  {sorter}実行完了: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} (経過時間: {elapsed_time/60:.1f}分)")

        print(f"  Analyzer作成開始: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        analyzer = si.create_sorting_analyzer(sorting=sorting, recording=recording, format='binary_folder', folder=dict_path["exp"] / sorter / name_thisparam / "analyzer", overwrite=True)
        print(analyzer)
        print("===== Sorting done =====")

    except Exception as e:
        print(f"Error occurred while running {sorter}: {e}")
        traceback.print_exc()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            gc.collect()
            print(f"  GPUメモリクリア完了 - 割り当て済み: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
else:
    print("skip sorting.")


In [ ]:
### export to phy ###
if do_export_phy:
    analyzer = si.load_sorting_analyzer(folder= dict_path["exp"] / sorter / name_thisparam / "analyzer")
    extensions = {
        'random_spikes': {},
        'waveforms': {},
        'templates': {},
        'spike_locations': {},
        'principal_components': {},
    }
    analyzer.compute(extensions)
    # PC特徴の計算が時間かかってそうなので、Falseにしておく
    sexp.export_to_phy(
        sorting_analyzer=analyzer, 
        output_folder=dict_path["exp"] / sorter / name_thisparam / "phy", 
        remove_if_exists=True,
        # compute_pc_features=False
        )
else:
    print("skip export to phy.")


In [ ]:
### convert to matlab files ###
convert_to_matlab_files(res_dir=dict_path["exp"] / sorter / name_thisparam, recording=recording)

### export to probe.json ###
write_probeinterface(dict_path["exp"] / sorter / name_thisparam / "probe.json", probe_or_probegroup=probe)

# stim_times_rise, stim_times_fall, fs_obx = get_stimtime(meta_obx, dict_path["obx"]["bin"])
# savemat(dict_path["exp"] / sorter / name_thisparam / "stim_times.mat", {'stim_times_rise': stim_times_rise, 'stim_times_fall': stim_times_fall, 'fs': fs_obx})


In [ ]:
# from dartsort.util.data_util import DARTsortSorting

# # .npzファイルからロード（HDF5も自動的に読み込まれる）
# sorting = DARTsortSorting.load(
#     r"C:\Users\tanaka-users\NeuronData\ge6w2\ge6w2_ep005_002\DARTsort\setting3\sorting\sorter_output\dartsort_sorting.npz"
# )

# # データにアクセス
# print(f"スパイク数: {sorting.n_spikes}")
# print(f"ユニット数: {sorting.n_units}")
# print(f"サンプリング周波数: {sorting.sampling_frequency}")

# # 基本データ
# times_samples = sorting.times_samples
# channels = sorting.channels
# labels = sorting.labels

# # 追加特徴量（HDF5から読み込まれたもの）
# if sorting.extra_features:
#     print(f"利用可能な特徴量: {list(sorting.extra_features.keys())}")
#     print(sorting.geom)

In [ ]:
# ### export to matlab files for ks4 ###
# conv_ks4_mat(res_dir=dict_path["exp"] / sorter / name_thisparam / "sorting" / "sorter_output", recording=recording)

# ### export to probe.json ###
# write_probeinterface(dict_path["exp"] / sorter / name_thisparam / "probe.json", probe_or_probegroup=probe)

# ### plot drift amount, scatter, diagnostics, spike positions ###
# ks_dir = dict_path["exp"] / sorter / name_thisparam / "sorting" / "sorter_output"
# ops, st, clu, similar_templates, \
#     is_ref, est_contam_rate, kept_spikes, \
#         tF, Wall, full_st, full_clu, full_amp = \
#             load_sorting(ks_dir, device="cuda", load_extra_vars=True)

# plot_drift_amount(ops, ks_dir)
# plot_drift_scatter(full_st, ks_dir)
# plot_diagnostics(Wall, full_clu, ops, ks_dir)
# plot_spike_positions(clu, is_ref, ks_dir)


# stim_times_rise, stim_times_fall, fs_obx = get_stimtime(meta_obx, dict_path["obx"]["bin"])
# savemat(dict_path["exp"] / sorter / name_thisparam / "stim_times.mat", {'stim_times_rise': stim_times_rise, 'stim_times_fall': stim_times_fall, 'fs': fs_obx})
